# 3.4 Calling differentially expressed peaks with DESeq2 and limma

### IMPORTANT: Please make sure that you are using the R kernel to run this notebook. ###


In this tutorial, we will focus on calling differential peaks: 
![Analysis pipeline](images/part4.png)

## Running DESeq

DESeq(https://bioconductor.org/packages/release/bioc/html/DESeq2.html) uses read count data, such as in our matrix **all.readcount.txt**, to estimate differential gene expression across conditions specified in a metdata file.  We run DESeq with the following comparisons (which we call "contrasts"): 

* 0min WT vs 45min WT 
* Timepoint comparisons 
    * MSN1 (0min vs 45min) 
    * MSN2 (0min vs 45min)
    * MSN4 (0min vs 45min)
    * HOG1 (0min vs 45min)
    * HOT1 (0min vs 45min)
    * YAP1 (0min vs 45min)
    * YAP6 (0min vs 45min)
    * YAP7 (0min vs 45min)
    
* Strain vs WT at 45min: 
    *  WT vs MSN1 (45min)
    *  WT vs MSN2 (45min)
    *  WT vs MSN4 (45min)
    *  WT vs HOG1 (45min)
    *  WT vs SKN7 (45min)
    *  WT vs HOT1 (45min)
    *  WT vs YAP1 (45min)
    *  WT vs YAP6 (45min)
    *  WT vs YAP7 (45min)  

In [ ]:
#change to your working directory 
username="annashch"
setwd(paste("/scratch/",username,sep=""))

In [ ]:
#load the DESeq2 library
library(DESeq2,quietly = TRUE)


In [ ]:
#We read in the counts data matrix and the metdata matrix in the same manner as we did in tutorial 3.1 
#load the read count matrix
count_data=read.table("/outputs/summary_pilot/all.readcount.txt",header=TRUE)
rownames(count_data)=paste(count_data$Chrom,count_data$Start,count_data$End,sep='\t')
#remove the columns we will not use 
count_data$Chrom=NULL
count_data$Start=NULL
count_data$End=NULL
count_data$ID=NULL

head(count_data)

In [ ]:
ncol(count_data)

In [ ]:
metadata=read.table("/metadata/TC2020_samples.tsv",header=TRUE)
#We use the "factor" function to tell R which variables are categorical rather than continuous 
metadata$Strain=factor(metadata$Strain)
metadata$Timepoint=factor(metadata$Timepoint,levels=c("45min","0min"))
metadata$GeneratedByResearcher=factor(metadata$GeneratedByResearcher)
#we don't need the other metadata columns for this analysis 
metadata$Sample=NULL
metadata$Replicate=NULL
metadata$GeneratedByGroup=NULL
metadata$AssignedTo=NULL
#we modify the ID column by pre-facing it with an X to generate valid R index names (this column will be used as an index column in PCA analysis)
metadata$ID=make.names(metadata$ID)

rownames(metadata)=metadata$ID
metadata$ID=NULL
#make sure the rows in metadata match the order of the columns in count_data 
metadata=metadata[names(count_data),]
metadata

In [ ]:
#we verify that 45min is the "base" timepoint
metadata$Timepoint

In [ ]:
#We set threshold for determining differential expression 
padjust_thresh=0.01 


## Analysis 1: WT 0min vs 45 min


We first subset our count_data and metadata to contain just the WT samples. 

In [ ]:
wt_count_data=count_data[,grep( "WT", names( count_data ))]

In [ ]:
head(wt_count_data)

In [ ]:
wt_metadata=metadata[metadata$Strain=="WT",]
wt_metadata

In [ ]:
#create a DESeq2 object with the data, metadata, and model information 
wt_ddsMat=DESeqDataSetFromMatrix(countData=as.matrix(wt_count_data),
                            colData=wt_metadata,
                            design=~Timepoint)


In [ ]:
#Run DESeq2 analysis 
wt_dds<-DESeq(wt_ddsMat)

In [ ]:
#We can examine several contrasts in the resulting DESeq2 object
resultsNames(wt_dds)

In [ ]:
#Specify the contrast we want to examine (0min vs 45min)
wt_ds=results(wt_dds,contrast=c("Timepoint","0min","45min"))

In [ ]:
wt_ds

In [ ]:
#subset the peak set to just the differential peaks 
wt_ds=na.omit(wt_ds)
sig=wt_ds[wt_ds$padj<padjust_thresh,] 
    
#find positive log fold change peaks 
positive_sig=sig[sig$log2FoldChange > 0,]
    
#find negative log fold change peaks 
negative_sig=sig[sig$log2FoldChange <0,]
    


In [ ]:
#peaks stronger at 0 min relative to 45 min 
positive_sig

In [ ]:
#peaks stronger at 45min relative to 0 min
negative_sig

In [ ]:
#write the significant peaks to output files 
write.table(row.names(positive_sig),
            file="WT_0min_vs_45min.positive.pilot.txt",
            quote=FALSE,row.names=FALSE,col.names=FALSE,sep='\t')

write.table(row.names(negative_sig),
            file="WT_0min_vs_45min.negative.pilot.txt",
            quote=FALSE,row.names=FALSE,col.names=FALSE,sep='\t')

## Analysis 2: 0min vs 45min within TF knockout strains 

Now, we replace the "WT" strain with a knockout strain and re-run the differential analysis. 

In [ ]:
tfs=c("MSN1","MSN2","MSN4", "YAP1", "YAP6", "YAP7", "HOG1", "HOT1", "SKN7")

for(tf in tfs){
    print(tf)
    cur_tf=tf
    tf_count_data=count_data[,grep( cur_tf, names( count_data ))]
    tf_metadata=metadata[metadata$Strain==cur_tf,]

    #create a DESeq2 object with the data, metadata, and model information 
    tf_ddsMat=DESeqDataSetFromMatrix(countData=as.matrix(tf_count_data),
                            colData=tf_metadata,
                            design=~Timepoint)

    #Run DESeq2 analysis 
    tf_dds<-DESeq(tf_ddsMat)

    tf_ds=results(tf_dds,contrast=c("Timepoint","0min","45min"))

    #subset the peak set to just the differential peaks 
    tf_ds=na.omit(tf_ds)
    sig=tf_ds[tf_ds$padj<padjust_thresh,] 
    
    #find positive log fold change peaks 
    positive_sig=sig[sig$log2FoldChange > 0,]
    print(positive_sig) 

    #find negative log fold change peaks 
    negative_sig=sig[sig$log2FoldChange <0,]
    print(negative_sig)

    #write the significant peaks to output files 
    write.table(row.names(positive_sig),
            file=paste(cur_tf,"0min_vs_45min.positive.pilot.txt",sep='_'),
            quote=FALSE,row.names=FALSE,col.names=FALSE,sep='\t')

    write.table(row.names(negative_sig),
            file=paste(cur_tf,"0min_vs_45min.negative.pilot.txt",sep="_"),
            quote=FALSE,row.names=FALSE,col.names=FALSE,sep='\t')
}

## Analysis 3: KO Strain vs WT at 45min

Now we will find differential peaks between the TF knockout strains and WT at 45 mins. We use the full data frame and metadata frame for this analysis.

In [ ]:
#create a DESeq2 object with the data, metadata, and model information 
ddsMat=DESeqDataSetFromMatrix(countData=as.matrix(count_data),
                            colData=metadata,
                            design=~Strain)

#Run DESeq2 analysis 
dds<-DESeq(ddsMat)

In [ ]:
#Specify the contrasts we want to examine (TF KO strain vs WT at 45 mins)
#Note, above, we set 45min as the first factor level for the Timeponit variable, so Deseq will perform the 
# Strain vs WT comparison at 45min. 
# If we had run factor(count_data$Timepoint,levels=c("0min","45min"))
deseq_contrasts=list(c("Strain","WT","MSN1"),
                     c("Strain","WT","MSN2"),
                     c("Strain","WT","MSN4"),
                     c("Strain","WT","HOG1"),
                     c("Strain","WT","SKN7"),
                     c("Strain","WT","HOT1"),
                     c("Strain","WT","YAP1"),
                     c("Strain","WT","YAP6"),
                     c("Strain","WT","YAP7"))
contrast_names=c("Strain_WT_vs_MSN1",
        "Strain_WT_vs_MSN2",
        "Strain_WT_vs_MSN4",
        "Strain_WT_vs_HOG1",
        "Strain_WT_vs_SKN7",
        "Strain_WT_vs_HOT1",
        "Strain_WT_vs_YAP1",
        "Strain_WT_vs_YAP6",
        "Strain_WT_vs_YAP7")



In [ ]:
#Query the DESeq2 results to find differential peaks for each contrast, using our padjust_thresh and lfc_thresh values.
for(contrast_index in seq(1,length(deseq_contrasts)))
{
        comparison_name=unlist(contrast_names[contrast_index])    
        print(comparison_name)
        ds=results(dds,
           contrast=unlist(deseq_contrasts[contrast_index]))
       
    
        #subset the peak set to just the differential peaks 
        ds=na.omit(ds)
        sig=ds[ds$padj<padjust_thresh,] 
    
        #find positive log fold change peaks 
        positive_sig=sig[sig$log2FoldChange > 0,]
    
        #find negative log fold change peaks 
        negative_sig=sig[sig$log2FoldChange <0,]
    
        write.table(row.names(positive_sig),
                    file=paste(comparison_name,".differential.positive.pilot.txt",sep=""),
                    quote=FALSE,sep='\t',row.names=FALSE,col.names=FALSE)
        write.table(row.names(negative_sig),
                    file=paste(comparison_name,".differential.negative.pilot.txt",sep=""),
                    quote=FALSE,sep='\t',row.names=FALSE,col.names=FALSE)
}


We now have the following differential comparison outputs from DESEQ2:

| Differential Peaks | File                                              |
|--------------------|---------------------------------------------------|
| 158                | SKN7_0min_vs_45min.negative.pilot.txt             |
| 117                | YAP7_0min_vs_45min.negative.pilot.txt             |
| 91                 | HOT1_0min_vs_45min.negative.pilot.txt             |
| 76                 | WT_0min_vs_45min.negative.pilot.txt               |
| 73                 | MSN1_0min_vs_45min.negative.pilot.txt             |
| 68                 | WT_0min_vs_45min.positive.pilot.txt               |
| 64                 | MSN2_0min_vs_45min.negative.pilot.txt             |
| 64                 | YAP6_0min_vs_45min.negative.pilot.txt             |
| 63                 | YAP1_0min_vs_45min.negative.pilot.txt             |
| 61                 | YAP7_0min_vs_45min.positive.pilot.txt             |
| 57                 | MSN4_0min_vs_45min.negative.pilot.txt             |
| 54                 | HOG1_0min_vs_45min.negative.pilot.txt             |
| 49                 | MSN2_0min_vs_45min.positive.pilot.txt             |
| 45                 | YAP6_0min_vs_45min.positive.pilot.txt             |
| 39                 | MSN1_0min_vs_45min.positive.pilot.txt             |
| 38                 | SKN7_0min_vs_45min.positive.pilot.txt             |
| 36                 | MSN4_0min_vs_45min.positive.pilot.txt             |
| 25                 | HOT1_0min_vs_45min.positive.pilot.txt             |
| 24                 | HOG1_0min_vs_45min.positive.pilot.txt             |
| 18                 | Strain_WT_vs_YAP6.differential.negative.pilot.txt |
| 11                 | YAP1_0min_vs_45min.positive.pilot.txt             |
| 7                  | Strain_WT_vs_YAP6.differential.positive.pilot.txt |
| 5                  | Strain_WT_vs_SKN7.differential.positive.pilot.txt |
| 4                  | Strain_WT_vs_SKN7.differential.negative.pilot.txt |
| 3                  | Strain_WT_vs_MSN2.differential.positive.pilot.txt |
| 2                  | Strain_WT_vs_HOT1.differential.positive.pilot.txt |
| 2                  | Strain_WT_vs_MSN1.differential.positive.pilot.txt |
| 2                  | Strain_WT_vs_YAP1.differential.negative.pilot.txt |
| 2                  | Strain_WT_vs_YAP1.differential.positive.pilot.txt |
| 1                  | Strain_WT_vs_HOG1.differential.negative.pilot.txt |
| 1                  | Strain_WT_vs_HOT1.differential.negative.pilot.txt |
| 1                  | Strain_WT_vs_MSN1.differential.negative.pilot.txt |
| 1                  | Strain_WT_vs_MSN2.differential.negative.pilot.txt |
| 1                  | Strain_WT_vs_MSN4.differential.negative.pilot.txt |
| 1                  | Strain_WT_vs_MSN4.differential.positive.pilot.txt |
| 1                  | Strain_WT_vs_YAP7.differential.negative.pilot.txt |
| 1                  | Strain_WT_vs_YAP7.differential.positive.pilot.txt |
| 0                  | Strain_WT_vs_HOG1.differential.positive.pilot.txt |


The output files contain chromosome positions of open peaks from ATAC‐seq. The p‐value cutoff for differential openness that we use is 0.01. 

### Running limma ###

If you recall, we used the R limma package to remove the "Researcher" batch effect in our data. Limma can also be used for differential peak calling. Limma uses a similar algorithm to DESeq2. We will go through the process of calling differential peaks with limma and see how the peak rankings differ between limma and DESeq2 -- it's always best to sanity check your results by running them through several similar analysis algorithms. For the sake of time, we will only reproduce "Analysis 1" -- examining the number of differential peaks in the WT strain between 45 mins and 0 mins. 

In [ ]:
#import the limma library 

library(limma)
#design the model 
design=model.matrix(~0+Timepoint,data=wt_metadata)

#We use the "voom" function associated with the limma package to normalize the count data 
vm=voom(wt_count_data,design)

#fit the model to the data 
fit=lmFit(vm,design=vm$design)




In [ ]:
cont.matrix=makeContrasts(timepoint="Timepoint45min-Timepoint0min",levels=fit)
bayes_model=eBayes(contrasts.fit(fit,cont.matrix))
res_limma=topTable(bayes_model,n=nrow(count_data))
head(res_limma)

### Comparing DESeq2 and limma voom outputs ### 

In [ ]:
#wt_ds stores the deseq2 p-values for teh 0min vs 45min WT comparison 
wt_ds=results(wt_dds,contrast=c("Timepoint","0min","45min"))
res_deseq2=as.data.frame(wt_ds)
head(res_deseq2)


In [ ]:
#We need to merge the two result dataframes by peak name So that we can generate a scatterplot of
#padj in one vs the other 
res_limma$peak=rownames(res_limma)
res_deseq2$peak=rownames(res_deseq2)
nrow(res_limma)
nrow(res_deseq2)

In [ ]:
merged_df=merge(res_limma,res_deseq2,by="peak")
merged_df$deseq2_p=-10*log10(merged_df$pvalue)
merged_df$limma_p=-10*log10(merged_df$P.Value)



In [ ]:
head(merged_df)

In [ ]:
library(ggplot2)
ggplot(merged_df,aes(x=deseq2_p,y=limma_p))+
    geom_point(alpha=0.1)+
    geom_abline()+
    xlim(0,200)+  
    ylim(0,200)

The pvalues reported by the methods are correlated. 

In [ ]:
spearman_cor=cor(merged_df$limma_p,merged_df$deseq2_p,method="spearman",use="complete.obs")
spearman_cor

In [ ]:
pearson_cor=cor(merged_df$limma_p,merged_df$deseq2_p,method="pearson",use="complete.obs")
pearson_cor

Finally, we plot the rank comparison of the p-values across the two methods. 

In [ ]:
#use the "rank" function to generate rank columns for the p-values 
merged_df$limma_p_rank=rank(merged_df$limma_p)
merged_df$deseq2_p_rank=rank(merged_df$deseq2_p)

ggplot(merged_df,aes(x=deseq2_p_rank,y=limma_p_rank))+
    geom_point(alpha=0.1)

In [ ]:
pearson_cor_rank=cor(merged_df$limma_p_rank,merged_df$deseq2_p_rank,method="pearson")
pearson_cor_rank